In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import stim

class BSCodeStimCircuit():

    def __init__(self,
                 d: int,
                 T: int,
                 xbasis: bool = False,
                 resets: bool = False,
                 bridge_qubits: bool = False,
                 ):
        
        if bridge_qubits == False:
            raise NotImplementedError("Bacon-Shor code without bridge qubits is not implemented yet.")
        if resets == False:
            raise NotImplementedError("Bacon-Shor code without resets is not implemented yet.")
        
        self.d = d
        self.T = T

        self._xbasis = xbasis
        self._resets = resets
        self._bridge_qubits = bridge_qubits

        # Qubit registers
        start, end = 0, d**2
        self.code_qubits = list(range(start, end))

        start, end = end, end+(d-1)*d
        self.x_ancilla_qubits = list(range(start, end))

        start, end = end, end+(d-1)*d
        self.z_ancilla_qubits = list(range(start, end))

        start, end = end, end+2*(d-1)*d
        self.bulk_bridge_qubits = list(range(start, end))

        start, end = end, end+2*(d-1)
        self.boundary_bridge_qubits = list(range(start, end))

        # Stim blocks
        self._Z_ent_block = stim.Circuit()
        self._X_ent_block = stim.Circuit()
        self._det_block = stim.Circuit()
        self.create_blocks()

        # Create the circuits
        self.circuits = {}
        for log in ["0", "1"]:
            self.circuits[log] = stim.Circuit()

        self._preparation()
        self._first_round()
        if resets == False:
            self._first_round()
        self._subsequent_rounds()
        self._final_readout()

    def measure(self, circuit: stim.Circuit, targets: list, p_soft: float = 0, p_hard: float = 0):
        readout = 'MR' if self._resets else 'M'
        circuit.append('X_ERROR', targets, arg=p_hard) if p_hard > 0 else None
        circuit.append(readout, targets, arg=p_soft)
        if self._resets: # reset errors ≈ readout_error for active resets
            circuit.append('X_ERROR', targets, arg=(p_soft + p_hard)) if (p_soft + p_hard) > 0 else None 

    def _x(self, logs=("0", "1")):
        for log in logs:
            if self._xbasis:
                targets = self.code_qubits[:self.d]
                self.circuits[log].append('Z', targets)
                self.circuits[log].append('TICK')
            else:
                targets = self.code_qubits[list(range(0, len(self.code_qubits), self.d))]
                self.circuits[log].append('X', targets)
                self.circuits[log].append('TICK')

    def _preparation(self):
        for log in ["0", "1"]:
            targets = self.code_qubits + self.x_ancilla_qubits + self.z_ancilla_qubits
            targets += self.bulk_bridge_qubits + self.boundary_bridge_qubits
            self.circuits[log].append('R', targets)
            if self._xbasis:
                self.circuit[log].append('H', self.code_qubits)
            self.circuits[log].append('TICK')
        self._x(["1"])
    
    def _first_round(self):
        rec = stim.target_rec
        for log in ["0", "1"]:
            self.circuits[log] += self._Z_ent_block
            self.circuits[log] += self._X_ent_block
            # assuming the detector blocks also has the msmts!
            # Measure Z
            targets = self.z_ancilla_qubits 
            self.circuits[log].append('MR', targets)
            for i in range(self.d-1):
                n_za = len(self.z_ancilla_qubits)
                targets = list(range(-i, -(len(self.code_qubits)+i), self.d-1))
                targets = list(range(-n_za+i, -n_za+(len(self.z_ancilla_qubits)+i), self.d-1))
                self.circuits[log].append('DETECTOR', [rec(i) for i in targets])

            # Measure X


        





In [6]:
import stim

circ = stim.Circuit()